<a href="https://colab.research.google.com/github/Prabhu-Tejas/GenAIclimbers/blob/main/GenAiclimbers_v1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the Generative AI Hackthon Project.

Team name: GenAI Climbers


Project name: Insurance:Expidating document validation processing.

In [12]:
# Base system dependencies
!sudo apt -y -qq install tesseract-ocr libtesseract-dev

# required by PyPDF2 for page count and other pdf utilities
!sudo apt-get -y -qq install poppler-utils python-dev libxml2-dev libxslt1-dev antiword unrtf poppler-utils pstotext tesseract-ocr flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig

# Python dependencies
!pip install google-cloud-aiplatform pytesseract PyPDF2 textract --upgrade --quiet --user


The following additional packages will be installed:
  libarchive-dev libleptonica-dev tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev tesseract-ocr
  tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 6 newly installed, 0 to remove and 15 not upgraded.
Need to get 8,193 kB of archives.
After this operation, 31.9 MB of additional disk space will be used.
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 6.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 123125 files and directories

In [13]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
!pip3 install PyPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.8/254.8 kB 5.7 MB/s eta 0:00:00


In [2]:
!pip install langchain

In [1]:
!apt-get install python3-pypdf2


Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-pypdf2 is already the newest version (1.26.0-3ubuntu1.20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [3]:
#importing Libraries

import urllib
import warnings
from pathlib import Path as p
from pprint import pprint

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

warnings.filterwarnings("ignore")
import PyPDF2

In [4]:
from google.colab import auth
auth.authenticate_user()

In [5]:
PROJECT_ID = "gen-genaiclimbers"

import vertexai

vertexai.init(project=PROJECT_ID, location="us-west1")

In [6]:
vertex_llm_text = VertexAI(model_name="text-bison@001")
vertex_embeddings = VertexAIEmbeddings(model_name="textembedding-gecko@001")



PROJECT_ID = "gen-genaiclimbers"  # @param {type:"string"}
REGION = "us-west1"

import vertexai

vertexai.init(project=PROJECT_ID, location=REGION)

vertex_llm_text = VertexAI(model_name="text-bison@001")
vertex_embeddings = VertexAIEmbeddings(model_name="textembedding-gecko@001")

In [12]:
#data ingestion: Please upload the pdf file.

pdf_loader = PyPDFLoader('ShahRukh_MedicalReport.pdf')
pages = pdf_loader.load_and_split()
print(pages[3].page_content)

PdfStreamError: ignored

In [ ]:
#Prompting

question = "Verify the hospital report, doctor report and the patient name and tell us whether the insurance claim can be approved?"
prompt_template = """Answer the question as precise as possible using the provided context. If the report is
                    not sufficient in the context, say "Claim is rejected" \n\n
                    Verify the doctor report \n
                    Verify the hospital report \n
                    Verify the names in doctor report , hospital report and verify the patient name
                    and tell in precise way\n
                    and display the patient name and company name \n
                    will the cost claimed in the insurance policy \n
                    Context: \n {context}?\n
                    Question: \n {question} \n
                    Answer:
                  """

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
# we are using Stuffing type of LLM chain

stuff_chain = load_qa_chain(vertex_llm_text, chain_type="stuff", prompt=prompt)

Stuffing Chain

The Stuffing chain serves as a solution for scenarios where the context length of the LLM is inadequate to handle extensive documents or a substantial amount of information. In such cases, a large document can be divided into smaller segments, and semantic search techniques can be employed to retrieve relevant documents based on the query. These retrieved documents are then “stuffed” into the LLM context, allowing for the generation of a response.

In [ ]:
from datetime import datetime
start = datetime.now()
stuff_answer = stuff_chain(
    {"input_documents": pages[1:4], "question": question}, return_only_outputs=True
)
end = datetime.now()
td = (end - start).total_seconds() * 10**3
print(f"The time of execution of above program is : {td:.03f}ms")

The time of execution of above program is : 1539.219ms


In [ ]:
pprint(stuff_answer)

{'output_text': "The insurance claim can be approved. The patient's name is "
                'Shah Rukh, and they are covered under the Capgemini group '
                'insurance policy. The surgery is necessary to repair and '
                "stabilize the fractured bone in the patient's right hand. The "
                'estimated cost of the surgery is INR 1,10,000/-, and the '
                'insurance policy covers this cost.'}
